**Teleportation and entanglement: Moving quantum data around**

In this notebook we focus on ❶ Moving data around a quantum computer using classical and quantum control. ❷ Visualizing single-qubit operations with the Bloch sphere and ❸ Predicting the output of two-qubit operations, and Pauli operations.

## Setup

In [ ]:
!pip install qutip

     |████████████████████████████████| 14.6 MB 4.7 MB/s 


## Swap in Python

In [ ]:
# SWAP: Using QuTiP’s swap on |+0 〉 to get the |0+〉 state
import qutip as qt
from qutip.qip.operations import hadamard_transform

ket_0 = qt.basis(2,0)
ket_plus = hadamard_transform() * ket_0
initial_state = qt.tensor(ket_plus, ket_0)
print(initial_state)

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.70710678]
 [0.        ]]


In [ ]:
swap_matrix = qt.swap() # Gets a copy of the unitary matrix for the swap instruction by calling qt.swap
print(swap_matrix * initial_state)
# When we do so, we end up in a superposition between |00 〉 and |01 〉 instead of between |00 〉 and |10 〉.

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]
 [0.        ]
 [0.        ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  """Entry point for launching an IPython kernel.


In [ ]:
print(qt.tensor(ket_0, ket_plus))

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]
 [0.        ]
 [0.        ]]


In particular, `swap` took two qubits that started in the state $|+0 〉$ to the $|0+〉$ state. More generally, we can read what the swap instruction does by looking at the unitary matrix we used to simulate it.

In [ ]:
# Unitary matrix for the swap instruction
print(qt.swap())

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  


If a matrix acts on a single-qubit register, we can use QuTiP to apply that to a register with an arbitrary number of qubits by using the `gate_expand _1toN` function . This takes the tensor product of identity operators on each qubit except the qubits we’re working with. In the same way, we can call QuTiP’s `gate_expand_2toN` function to turn two-qubit unitary matrices into matrices that we can use to simulate how two-qubit operations like swap transform the state of a whole register. Let’s add that to our simulator.

In [ ]:

from interface import QuantumDevice, Qubit
import qutip as qt
from qutip.qip.operations import hadamard_transform
import numpy as np
from typing import List

KET_0 = qt.basis(2, 0)
H = hadamard_transform()

class SimulatedQubit(Qubit):
    qubit_id: int
    parent: "Simulator"

    def __init__(self, parent_simulator: "Simulator", id: int):
        self.qubit_id = id
        self.parent = parent_simulator

    def h(self) -> None:
        self.parent._apply(H, [self.qubit_id])

    def measure(self) -> bool:
        projectors = [
            qt.circuit.gate_expand_1toN(
                qt.basis(2, outcome) * qt.basis(2, outcome).dag(),
                self.parent.capacity,
                self.qubit_id
            )
            for outcome in (0, 1)
        ]
        post_measurement_states = [
            projector * self.parent.register_state
            for projector in projectors
        ]
        probabilities = [
            post_measurement_state.norm() ** 2
            for post_measurement_state in post_measurement_states
        ]
        sample = np.random.choice([0, 1], p=probabilities)
        self.parent.register_state = post_measurement_states[sample].unit()
        return int(sample)

    def reset(self) -> None:
        if self.measure(): self.x()

    def swap(self, target: Qubit) -> None:
        self.parent._apply(
            qt.swap(),
            [self.qubit_id, target.qubit_id]
        )

    def cnot(self, target: Qubit) -> None:
        self.parent._apply(
            qt.cnot(),
            [self.qubit_id, target.qubit_id]
        )

    def rx(self, theta: float) -> None:
        self.parent._apply(qt.rx(theta), [self.qubit_id])

    def ry(self, theta: float) -> None:
        self.parent._apply(qt.ry(theta), [self.qubit_id])

    def rz(self, theta: float) -> None:
        self.parent._apply(qt.rz(theta), [self.qubit_id])

    def x(self) -> None:
        self.parent._apply(qt.sigmax(), [self.qubit_id])

    def y(self) -> None:
        self.parent._apply(qt.sigmay(), [self.qubit_id])

    def z(self) -> None:
        self.parent._apply(qt.sigmaz(), [self.qubit_id])

class Simulator(QuantumDevice):
    capacity: int
    available_qubits: List[SimulatedQubit]
    register_state: qt.Qobj

    def __init__(self, capacity=3):
        self.capacity = capacity
        self.available_qubits = [
            SimulatedQubit(self, idx)
            for idx in range(capacity)
        ]
        self._sort_available()
        self.register_state = qt.tensor(
            *[
                qt.basis(2, 0)
                for _ in range(capacity)
            ]
        )

    def _sort_available(self) -> None:
        self.available_qubits = list(sorted(
            self.available_qubits,
            key=lambda qubit: qubit.qubit_id,
            reverse=True
        ))

    def allocate_qubit(self) -> SimulatedQubit:
        if self.available_qubits:
            return self.available_qubits.pop()

    def deallocate_qubit(self, qubit: SimulatedQubit):
        self.available_qubits.append(qubit)
        self._sort_available()

    def _apply(self, unitary: qt.Qobj, ids: List[int]):
        if len(ids) == 1:
            matrix = qt.circuit.gate_expand_1toN(unitary,                                                 self.capacity, ids[0])        elif len(ids) == 2:            matrix = qt.circuit.gate_expand_2toN(unitary,                                                 self.capacity, *ids)        else:            raise ValueError("Only one- or two-qubit unitary matrices supported.")
        self.register_state = matrix * self.register_state

    def dump(self) -> None:
        print(self.register_state)

## A teleportation program in Python

In [ ]:
#from interface import QuantumDevice, Qubit
#from simulator import Simulator

def teleport(msg: Qubit, here: Qubit, there: Qubit) -> None:
    here.h()
    here.cnot(there)

    # ...
    msg.cnot(here)
    msg.h()

    if msg.measure(): there.z()
    if here.measure(): there.x()

    msg.reset()
    here.reset()

if __name__ == "__main__":
    sim = Simulator(capacity=3)
    with sim.using_register(3) as (msg, here, there):
        msg.ry(0.123)
        teleport(msg, here, there)
        there.ry(-0.123)
        sim.dump()

## Running the simulator:

Now we can write a program to prepare two qubits in an entangled pair:

In [ ]:
from simulator import Simulator

sim = Simulator(capacity=2)
     with sim.using_register(2) as (you, eve):
     eve.h()
     eve.cnot(you)
     sim.dump()

❶ The teleport function takes two qubits as input: the qubit we want to move (msg) and where we want it to be moved (“there”). We also need one temporary qubit, which we call “here”. We presume by convention that both “here” and “there” start in the |0 〉 state. 

❷ We need to start with some entanglement between “here” and “there”. We can use our old friend, the h instruction, together with our new friend, the cnot instruction. 

❸ The only instruction in this program that needs to act on both “here” and “there”. After running this, we can send Eve our qubit, and both of us can run the rest of the program with only classical communication. 

❹ At this point in the program, “here” and “there” are in the (|00 〉 + |11 〉) / √2 state that we first saw in chapter 4. 

❺ Runs the program we used to prepare the (|00 〉 + |11 〉) / √2 state backward, but on the msg and “here” qubits that live entirely on our device. We can think of running a preparation backward as a kind of measurement, such that these steps set us up to measure the quantum message we’re trying to send Eve in an entangled basis. 

❻ When we actually do that measurement, we get classical data to send Eve. Once she has that data, she can use the x and z instructions to decode the quantum message. 

❼ Now that we’re done with our qubits, it’s good to put them back into |0 〉 so they’re ready to be used again. This doesn’t affect the state of “there”, though, as we’ve only reset our qubits, not the one we gave to Eve!

In [4]:
# Copyright (c) Sarah Kaiser and Chris Granade. Code sample from the book "Learn Quantum Computing with Python and Q#" by Sarah Kaiser and Chris Granade, published by Manning Publications Co. Book ISBN 9781617296130. Code licensed under the MIT License.